<a href="https://colab.research.google.com/github/aboagyeso/ML_Xgboost/blob/master/Copy_xgboost__Kinase_Regression_O75116.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT PACKAGES

In [0]:
import numpy as np
import sklearn.metrics, math
from sklearn.linear_model import LinearRegression  
from xgboost import XGBRegressor
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import KFold
from sklearn import metrics
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tensorflow import set_random_seed
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
# Set random seed
np.random.seed(0)

In [0]:

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error 
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
#from xgboost import XGBRegressor

DEFINE CUSTOM FUNCTIONS

In [0]:


# root mean squared error (rmse) for regression
def rmse(y_true, y_pred):
    from keras import backend
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

# mean squared error (mse) for regression
def mse(y_true, y_pred):
    from keras import backend
    return backend.mean(backend.square(y_pred - y_true), axis=-1)

# coefficient of determination (R^2) for regression
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

def r_square_loss(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return 1 - ( 1 - SS_res/(SS_tot + K.epsilon()))

In [0]:

def add_gausian(y,mu,sigma):
    noise = np.random.normal(mu,sigma,[1105, 117])
    df_aug = df_train_set +noise 
    #df_new = pd.concat([df_train_set,df_aug])
    return(df_aug)

def drop_corr(df):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column]>0.95)]
    df_new = df.drop(df[to_drop],axis = 1)
    return(df_new)

def model_eval(model):
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    #model_score = model.score(X_test,y_test)
    model_mse =  mse(y_test,pred)
    model_r2 =r2(y_test,pred)
    return(-model_mse, model_r2)

In [0]:
# Load libraries
from keras import models
from keras import layers
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

In [0]:

# imports
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import KFold
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor as xgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

READ IN THE DATAFRAME AND CHECK OUTPUT

In [32]:
df = pd.read_csv('https://raw.githubusercontent.com/aboagyeso/Kinase_activity/master/data/kinase_files/KinaseO75116.csv')
df.head()

,Unnamed: 0,target_id,pAc,smiles,SlogP,SMR,LabuteASA,TPSA,AMW,ExactMW,NumLipinskiHBA,NumLipinskiHBD,NumRotatableBonds,NumHBD,NumHBA,NumAmideBonds,NumHeteroAtoms,NumHeavyAtoms,NumAtoms,NumStereocenters,NumUnspecifiedStereocenters,NumRings,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,NumAromaticHeterocycles,NumSaturatedHeterocycles,NumAliphaticHeterocycles,NumAromaticCarbocycles,NumSaturatedCarbocycles,NumAliphaticCarbocycles,FractionCSP3,Chi0v,Chi1v,Chi2v,Chi3v,Chi4v,Chi1n,Chi2n,Chi3n,...,MQN2,MQN3,MQN4,MQN5,MQN6,MQN7,MQN8,MQN9,MQN10,MQN11,MQN12,MQN13,MQN14,MQN15,MQN16,MQN17,MQN19,MQN20,MQN21,MQN22,MQN23,MQN24,MQN25,MQN26,MQN27,MQN28,MQN29,MQN30,MQN31,MQN32,MQN33,MQN34,MQN35,MQN36,MQN37,MQN38,MQN39,MQN40,MQN41,MQN42
0,1381,O75116,2.49,COC1=C(C=C(C=C1)CC(=O)NC2=CC=CC(=C2)C(=O)NCCC3...,3.00450,120.8656,183.061693,88.69,425.529,425.231456,7,3,9,3,5,2,7,31,62,1,1,3,2,1,1,0,1,1,2,0,0,0.416667,18.201543,10.684906,7.752047,5.458231,3.781632,10.684906,7.752047,5.458231,...,0,0,0,0,0,0,2,1,4,0,31,13,2,0,12,6,9,11,7,3,3,0,0,4,7,2,0,12,6,0,0,0,0,3,0,0,0,0,0,0
1,1382,O75116,2.77,CN(C)CC1=CC=C(C=C1)NC(=O)C2=CC(=CC=C2)CNC(=O)C...,3.80218,120.5554,181.675368,85.23,412.493,412.189926,6,2,7,2,4,2,6,31,55,0,0,3,3,0,0,0,0,0,3,0,0,0.160000,17.553341,9.867272,7.461040,4.629035,3.064970,9.867272,7.461040,4.629035,...,0,0,0,0,0,0,4,0,2,0,31,12,2,1,9,9,7,8,6,2,2,0,0,5,5,3,0,12,6,0,0,0,0,3,0,0,0,0,0,0
2,1383,O75116,2.80,COC1=C(C=C(C=C1)NC(=O)NCC2=CC=CC(=C2)C(=O)NCCC...,3.14500,124.3903,188.611555,100.72,440.544,440.242356,8,4,9,4,5,3,8,32,64,1,1,3,2,1,1,0,1,1,2,0,0,0.416667,18.701543,10.934906,7.834772,5.520731,3.756409,10.934906,7.834772,5.520731,...,0,0,0,0,0,0,3,1,4,0,32,14,2,0,12,6,9,12,8,4,4,0,0,4,8,2,0,12,6,0,0,0,0,3,0,0,0,0,0,0
3,1384,O75116,2.89,CN(C)CC1=CC=C(C=C1)NC(=O)C2=CC(=CC=C2)CNC(=O)C...,3.93910,122.3924,182.396334,70.67,417.509,417.205242,6,2,8,2,4,2,6,31,58,0,0,3,3,0,0,0,0,0,3,0,0,0.200000,18.014375,10.006037,7.500387,4.705054,3.111225,10.006037,7.500387,4.705054,...,0,0,0,0,0,0,3,0,3,0,31,13,2,0,9,9,8,9,6,2,2,0,0,5,5,3,0,12,6,0,0,0,0,3,0,0,0,0,0,0
4,1385,O75116,3.16,COC1=C(C=C(C=C1)C(=O)NCC2=CC=CC(=C2)C(=O)NC3=C...,4.43430,126.8544,190.921474,89.55,441.487,441.168856,7,2,7,2,5,2,7,33,56,0,0,4,4,0,0,1,0,0,3,0,0,0.115385,18.292867,10.314616,7.344547,5.186407,3.448743,10.314616,7.344547,5.186407,...,0,0,0,0,0,0,2,1,4,0,33,11,2,0,12,11,7,11,7,2,2,0,0,4,5,2,0,14,8,0,0,0,0,4,0,0,0,0,2,1


In [33]:
#Check the shape of the dataframe
df.shape

(1228, 120)

In [34]:
#Drop the highly correlated features
drop_corr(df).head()

,Unnamed: 0,target_id,pAc,smiles,SlogP,SMR,TPSA,NumRotatableBonds,NumHBA,NumAmideBonds,NumStereocenters,NumUnspecifiedStereocenters,NumRings,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,NumAromaticHeterocycles,NumSaturatedHeterocycles,NumAliphaticHeterocycles,NumAromaticCarbocycles,NumSaturatedCarbocycles,NumAliphaticCarbocycles,FractionCSP3,HallKierAlpha,slogp_VSA1,slogp_VSA2,slogp_VSA3,slogp_VSA4,slogp_VSA5,slogp_VSA6,slogp_VSA7,slogp_VSA8,slogp_VSA10,slogp_VSA11,slogp_VSA12,smr_VSA1,smr_VSA2,smr_VSA3,smr_VSA4,smr_VSA5,...,peoe_VSA4,peoe_VSA5,peoe_VSA6,peoe_VSA7,peoe_VSA8,peoe_VSA9,peoe_VSA10,peoe_VSA11,peoe_VSA12,peoe_VSA13,peoe_VSA14,MQN2,MQN4,MQN5,MQN6,MQN7,MQN8,MQN9,MQN10,MQN11,MQN14,MQN15,MQN16,MQN24,MQN25,MQN26,MQN28,MQN29,MQN30,MQN31,MQN32,MQN33,MQN34,MQN35,MQN36,MQN37,MQN38,MQN39,MQN40,MQN41
0,1381,O75116,2.49,COC1=C(C=C(C=C1)CC(=O)NC2=CC=CC(=C2)C(=O)NCCC3...,3.00450,120.8656,88.69,9,5,2,1,1,3,2,1,1,0,1,1,2,0,0,0.416667,-3.06,25.424092,45.668224,11.215359,5.917906,35.183905,42.464569,0.0,0.000000,5.687386,11.499024,0.0,19.062800,0.000000,10.633577,5.917906,25.683286,...,0.000000,0.0,12.132734,74.165171,17.795594,20.640417,0.000000,11.499024,5.90718,5.907180,0.000000,0,0,0,0,0,2,1,4,0,2,0,12,0,0,4,2,0,12,6,0,0,0,0,3,0,0,0,0,0
1,1382,O75116,2.77,CN(C)CC1=CC=C(C=C1)NC(=O)C2=CC(=CC=C2)CNC(=O)C...,3.80218,120.5554,85.23,7,4,2,0,0,3,3,0,0,0,0,0,3,0,0,0.160000,-3.95,10.633577,30.809613,13.089513,11.331113,37.406332,72.796405,0.0,0.000000,5.687386,0.000000,0.0,9.589074,5.261892,10.216698,0.000000,13.089513,...,5.261892,0.0,24.265468,73.753183,29.903802,11.632673,0.000000,0.000000,0.00000,11.814359,0.000000,0,0,0,0,0,4,0,2,0,2,1,9,0,0,5,3,0,12,6,0,0,0,0,3,0,0,0,0,0
2,1383,O75116,2.80,COC1=C(C=C(C=C1)NC(=O)NCC2=CC=CC(=C2)C(=O)NCCC...,3.14500,124.3903,100.72,9,5,3,1,1,3,2,1,1,0,1,1,2,0,0,0.416667,-3.26,30.740880,45.792159,6.544756,5.917906,35.183905,42.464569,0.0,0.000000,10.481923,11.499024,0.0,19.062800,0.000000,15.950366,5.917906,25.807221,...,0.000000,0.0,12.132734,68.098803,30.406718,14.219595,0.000000,11.499024,0.00000,5.907180,6.031115,0,0,0,0,0,3,1,4,0,2,0,12,0,0,4,2,0,12,6,0,0,0,0,3,0,0,0,0,0
3,1384,O75116,2.89,CN(C)CC1=CC=C(C=C1)NC(=O)C2=CC(=CC=C2)CNC(=O)C...,3.93910,122.3924,70.67,8,4,2,0,0,3,3,0,0,0,0,0,3,0,0,0.200000,-3.64,15.370440,37.919411,13.089513,0.000000,31.842880,72.796405,0.0,0.000000,5.687386,5.749512,0.0,14.325937,0.000000,10.216698,0.000000,13.089513,...,0.000000,0.0,24.265468,73.753183,29.903802,7.109798,5.749512,0.000000,0.00000,11.814359,0.000000,0,0,0,0,0,3,0,3,0,2,0,9,0,0,5,3,0,12,6,0,0,0,0,3,0,0,0,0,0
4,1385,O75116,3.16,COC1=C(C=C(C=C1)C(=O)NCC2=CC=CC(=C2)C(=O)NC3=C...,4.43430,126.8544,89.55,7,5,2,0,0,4,4,0,0,1,0,0,3,0,0,0.115385,-4.39,20.107303,31.017933,6.544756,0.000000,26.279429,79.123725,0.0,10.772448,5.687386,11.499024,0.0,19.062800,0.000000,10.300767,0.000000,6.544756,...,0.000000,0.0,18.199101,59.480612,41.138957,14.219595,0.000000,11.499024,0.00000,11.814359,0.000000,0,0,0,0,0,2,1,4,0,2,0,12,0,0,4,2,0,14,8,0,0,0,0,4,0,0,0,0,2


In [35]:
drop_corr(df).shape

(1228, 87)

In [0]:
#Plot distribution of the pAc with the density estimation function
#f, ax = plt.subplots(figsize=(6, 6))
#sns.distplot(df.pAc,color = 'green');
#sns.set_style("whitegrid")

In [0]:
#Boxplot of pAc,It has a few outliers but the distribution in fairly normal
#f, ax = plt.subplots(figsize=(5, 6))
#sns.set_style("whitegrid")
#data = df.pAc
#sns.boxplot(data=data);

In [38]:
#Drop unwanted columns
df = df.drop(['Unnamed: 0','smiles','target_id'], axis=1)
df.head()

,pAc,SlogP,SMR,LabuteASA,TPSA,AMW,ExactMW,NumLipinskiHBA,NumLipinskiHBD,NumRotatableBonds,NumHBD,NumHBA,NumAmideBonds,NumHeteroAtoms,NumHeavyAtoms,NumAtoms,NumStereocenters,NumUnspecifiedStereocenters,NumRings,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,NumAromaticHeterocycles,NumSaturatedHeterocycles,NumAliphaticHeterocycles,NumAromaticCarbocycles,NumSaturatedCarbocycles,NumAliphaticCarbocycles,FractionCSP3,Chi0v,Chi1v,Chi2v,Chi3v,Chi4v,Chi1n,Chi2n,Chi3n,Chi4n,HallKierAlpha,kappa1,...,MQN2,MQN3,MQN4,MQN5,MQN6,MQN7,MQN8,MQN9,MQN10,MQN11,MQN12,MQN13,MQN14,MQN15,MQN16,MQN17,MQN19,MQN20,MQN21,MQN22,MQN23,MQN24,MQN25,MQN26,MQN27,MQN28,MQN29,MQN30,MQN31,MQN32,MQN33,MQN34,MQN35,MQN36,MQN37,MQN38,MQN39,MQN40,MQN41,MQN42
0,2.49,3.00450,120.8656,183.061693,88.69,425.529,425.231456,7,3,9,3,5,2,7,31,62,1,1,3,2,1,1,0,1,1,2,0,0,0.416667,18.201543,10.684906,7.752047,5.458231,3.781632,10.684906,7.752047,5.458231,3.781632,-3.06,22.621323,...,0,0,0,0,0,0,2,1,4,0,31,13,2,0,12,6,9,11,7,3,3,0,0,4,7,2,0,12,6,0,0,0,0,3,0,0,0,0,0,0
1,2.77,3.80218,120.5554,181.675368,85.23,412.493,412.189926,6,2,7,2,4,2,6,31,55,0,0,3,3,0,0,0,0,0,3,0,0,0.160000,17.553341,9.867272,7.461040,4.629035,3.064970,9.867272,7.461040,4.629035,3.064970,-3.95,21.751562,...,0,0,0,0,0,0,4,0,2,0,31,12,2,1,9,9,7,8,6,2,2,0,0,5,5,3,0,12,6,0,0,0,0,3,0,0,0,0,0,0
2,2.80,3.14500,124.3903,188.611555,100.72,440.544,440.242356,8,4,9,4,5,3,8,32,64,1,1,3,2,1,1,0,1,1,2,0,0,0.416667,18.701543,10.934906,7.834772,5.520731,3.756409,10.934906,7.834772,5.520731,3.756409,-3.26,23.404100,...,0,0,0,0,0,0,3,1,4,0,32,14,2,0,12,6,9,12,8,4,4,0,0,4,8,2,0,12,6,0,0,0,0,3,0,0,0,0,0,0
3,2.89,3.93910,122.3924,182.396334,70.67,417.509,417.205242,6,2,8,2,4,2,6,31,58,0,0,3,3,0,0,0,0,0,3,0,0,0.200000,18.014375,10.006037,7.500387,4.705054,3.111225,10.006037,7.500387,4.705054,3.111225,-3.64,22.054377,...,0,0,0,0,0,0,3,0,3,0,31,13,2,0,9,9,8,9,6,2,2,0,0,5,5,3,0,12,6,0,0,0,0,3,0,0,0,0,0,0
4,3.16,4.43430,126.8544,190.921474,89.55,441.487,441.168856,7,2,7,2,5,2,7,33,56,0,0,4,4,0,0,1,0,0,3,0,0,0.115385,18.292867,10.314616,7.344547,5.186407,3.448743,10.314616,7.344547,5.186407,3.448743,-4.39,21.827384,...,0,0,0,0,0,0,2,1,4,0,33,11,2,0,12,11,7,11,7,2,2,0,0,4,5,2,0,14,8,0,0,0,0,4,0,0,0,0,2,1


In [0]:
# Get the feature vector
X = df.drop(columns=['pAc'],axis=1)
# Get the target vector
y = df["pAc"]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=42)

In [41]:
#Check the shape of the training and testing data 
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

((1105, 116), (123, 116), (1105,), (123,))

 Add the X_train and y_train to form a a training set for augmentation
 **bold text**

In [42]:
df_train_set = pd.concat([X_train, y_train], axis = 1)
df_train_set.head()

,SlogP,SMR,LabuteASA,TPSA,AMW,ExactMW,NumLipinskiHBA,NumLipinskiHBD,NumRotatableBonds,NumHBD,NumHBA,NumAmideBonds,NumHeteroAtoms,NumHeavyAtoms,NumAtoms,NumStereocenters,NumUnspecifiedStereocenters,NumRings,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,NumAromaticHeterocycles,NumSaturatedHeterocycles,NumAliphaticHeterocycles,NumAromaticCarbocycles,NumSaturatedCarbocycles,NumAliphaticCarbocycles,FractionCSP3,Chi0v,Chi1v,Chi2v,Chi3v,Chi4v,Chi1n,Chi2n,Chi3n,Chi4n,HallKierAlpha,kappa1,kappa2,...,MQN3,MQN4,MQN5,MQN6,MQN7,MQN8,MQN9,MQN10,MQN11,MQN12,MQN13,MQN14,MQN15,MQN16,MQN17,MQN19,MQN20,MQN21,MQN22,MQN23,MQN24,MQN25,MQN26,MQN27,MQN28,MQN29,MQN30,MQN31,MQN32,MQN33,MQN34,MQN35,MQN36,MQN37,MQN38,MQN39,MQN40,MQN41,MQN42,pAc
1190,3.44740,143.4823,214.609051,109.46,495.583,495.227040,8,4,6,3,5,3,8,37,66,0,0,5,4,0,1,1,0,1,3,0,0,0.206897,20.875802,12.164143,9.534893,6.872022,4.775835,12.164143,9.534893,6.872022,4.775835,-4.43,24.272639,9.907351,...,0,0,0,0,0,3,2,3,0,37,11,3,0,16,11,6,11,8,4,3,0,0,6,3,3,0,14,11,0,0,0,1,4,0,0,0,0,4,2,9.59
836,2.83982,127.4709,192.698211,103.45,450.543,450.237939,9,2,6,2,7,1,9,33,63,0,0,4,3,1,1,2,1,1,1,0,0,0.416667,19.513235,11.010327,8.924242,6.309179,4.651698,11.010327,8.924242,6.309179,4.651698,-3.46,22.723924,9.187182,...,0,0,0,0,0,2,4,3,0,33,12,2,0,14,8,6,12,9,2,2,0,0,6,3,3,0,13,7,1,0,0,1,3,0,0,0,0,2,1,8.05
847,2.69480,96.9778,147.460646,100.45,349.369,349.133888,6,4,4,3,4,0,7,26,42,1,0,4,4,0,0,2,0,0,2,0,0,0.105263,13.738600,8.131362,6.111620,4.286360,2.900776,8.131362,6.111620,4.286360,2.900776,-3.32,16.164846,6.530382,...,0,0,0,0,0,1,4,1,0,26,6,1,0,11,11,4,7,6,4,3,0,0,3,1,1,0,13,8,0,0,0,1,3,0,0,0,0,2,1,8.10
933,3.43162,108.4881,161.299666,73.99,381.863,381.124405,5,3,3,3,3,1,6,27,47,2,2,4,3,1,1,1,1,1,2,0,0,0.238095,15.638106,9.296233,7.407923,5.452985,3.946743,8.918269,7.000723,5.140276,3.735834,-2.63,17.767379,7.175847,...,1,0,0,0,0,1,2,2,0,27,6,2,0,14,8,3,7,5,3,3,0,0,4,1,1,0,12,9,0,0,0,1,3,0,0,0,0,2,1,8.30
422,3.91990,96.7944,149.524987,67.01,353.809,353.093104,5,2,3,2,3,1,6,25,41,1,1,4,3,0,1,1,0,1,2,0,0,0.157895,14.085320,8.379827,6.454111,4.518750,3.125281,8.001863,6.017675,4.283653,2.930581,-2.79,15.721146,6.607182,...,1,0,0,0,0,1,2,1,1,25,5,1,0,14,8,3,7,5,2,2,0,0,2,1,1,0,14,7,0,0,0,1,3,0,0,0,0,2,1,6.88


In [0]:
#Assign df0 to df_train_set
df0 = df_train_set

In [44]:
#The shape of the training set
df0.shape

(1105, 117)

DATA BEFORE AUGMENTATION

In [45]:
# Get the feature vector
X0 = df0.drop(columns=['pAc'],axis=1).values
# Get the target vector
y0 = df0["pAc"].values
X0_train, X0_test, y0_train, y0_test = train_test_split(X0, y0, test_size=0.3,random_state=42)
X0_train.shape,y0_train.shape,X0_test.shape,y0_test.shape

((773, 116), (773,), (332, 116), (332,))

VALIDATION SET FOR EXTERNAL VALIDATION

In [0]:
X_test_val = X_test.values
y_test_val = y_test.values

AUGMENTATION 0.1

In [0]:
#Add noise of 0.01 to the dataframe
noise = add_gausian(df0,0,0.1)
df1 = pd.concat([df0,noise])
df1.shape
# Get the feature vector
X1 = df1.drop(columns=['pAc'],axis=1).values
# Get the target vector
y1 = df1["pAc"].values

#Split data into training and testing set
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3,random_state=42)

AUGMENTATION 0.2

In [0]:
#Add noise of 0.02 to the dataframe
noise = add_gausian(df0,0,0.2)
df2=pd.concat([df1,noise])

# Get the feature vector
X2 = df2.drop(columns=['pAc'],axis=1).values
# Get the target vector
y2 = df2["pAc"].values
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3,random_state=42)

AUGMENTATION 0.3

In [0]:
#Add noise of 0.03 to the dataframe
noise3 = add_gausian(df0,0,0.3)

df3 = pd.concat([df2,noise3])
# Get the feature vector
#X = df.drop('smiles', 'mol', 'pAc', axis=1)
X3 = df3.drop(columns=['pAc'],axis=1).values
# Get the target vector
y3 = df3["pAc"].values
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.3,random_state=42)

BUILD XGBOOST MODELS  MODELS


MODEL0

In [0]:
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}
xgb = XGBRegressor(nthread=-1) 
grid = GridSearchCV(xgb,params,cv=1)

In [51]:

#Train the model
model0=xgb.fit(X0_train, y0_train)



[17:54:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


MODEL 1

In [52]:

#Train the model
model1=xgb.fit(X1_train, y1_train) 

[17:54:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


MODEL 2

In [53]:

# fit model
model2 = xgb.fit(X2_train, y2_train)

[17:54:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


MODEL 3

In [54]:

# fit model
model3=xgb.fit(X3_train, y3_train)


[17:54:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


MODEL0 EVALUATION

In [55]:
pred0 =model0.predict(X0_test)
print('Model0 Evaluation on test set:')
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y0_test,pred0))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y0_test,pred0))
print("\n")
print('Model0 Evaluation on Validation set:')
valid_pred0= model0.predict(X_test_val)
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y_test_val,valid_pred0))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y_test_val,valid_pred0))


Model0 Evaluation on test set:
Mean squared error (MSE):       0.510085
R square (R^2):                 0.737677


Model0 Evaluation on Validation set:
Mean squared error (MSE):       1.133857
R square (R^2):                 0.545053


MODEL1 EVALUATION

In [56]:
pred1 =model1.predict(X1_test)
print('Model0 Evaluation on test set:')
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y1_test,pred1))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y1_test,pred1))
print("\n")
print('Model0 Evaluation on Validation set:')
valid_pred1= model1.predict(X_test_val)
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y_test_val,valid_pred1))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y_test_val,valid_pred1))

Model0 Evaluation on test set:
Mean squared error (MSE):       0.531867
R square (R^2):                 0.730609


Model0 Evaluation on Validation set:
Mean squared error (MSE):       1.133857
R square (R^2):                 0.545053


MODEL 2 EVALUATION

In [57]:
# Predict test set
print('Model2 Test:')
y2_pred = model2.predict(X2_test) 
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y2_test,y2_pred))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y2_test,y2_pred))
print("\n")
#Evaluate model2 on validation prediction
print('Model2 Validation:')
valid_pred2 = model2.predict(X_test_val)
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y_test_val,valid_pred2))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y_test_val,valid_pred2))



Model2 Test:
Mean squared error (MSE):       0.541872
R square (R^2):                 0.720784


Model2 Validation:
Mean squared error (MSE):       1.133857
R square (R^2):                 0.545053


MODEL 3 EVALUATION

In [58]:
# Predict test set
y3_pred = model3.predict(X3_test) 
print('Test Set:')
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y3_test,y3_pred))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y3_test,y3_pred))
print("\n")
#Evaluate model3 on validation prediction
print('Validation set:')
valid_pred3 = model3.predict(X_test_val)
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(y_test_val,valid_pred3))
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(y_test_val,valid_pred3))

Test Set:
Mean squared error (MSE):       0.652420
R square (R^2):                 0.682089


Validation set:
Mean squared error (MSE):       1.133857
R square (R^2):                 0.545053
